In [19]:
"""국토교통부 공동주택 기본정보 추출
- 링크: https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15058453
"""
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import io
import xmltodict
import json

# import xml.etree.ElementTree as et


# load .env
load_dotenv()
getRTMSDataSvcAptTradeDev_en = os.environ.get("getRTMSDataSvcAptTradeDev_en")
getRTMSDataSvcAptTradeDev_de = os.environ.get("pubkey_de")

In [20]:
df_apt = pd.read_csv("data/국토교통부_아파트단지목록_20220913.csv")

In [21]:
idx = (df_apt["as1"] == "서울특별시") & (df_apt["as2"] == "양천구")
YC_GU = df_apt[idx]

In [22]:
YC_GU

,as1,as2,as3,as4,bjdCode,kaptCode,kaptName
11,서울특별시,양천구,신정동,NaN,1147010100,A15886402,신정대성유니드아파트2차
213,서울특별시,양천구,신월동,NaN,1147010300,A15809402,신월삼정그린뷰
297,서울특별시,양천구,신정동,NaN,1147010100,A15807002,신트리1단지
503,서울특별시,양천구,신정동,NaN,1147010100,A15807316,신트리4단지
704,서울특별시,양천구,목동,NaN,1147010200,A15805301,목동부영그린타운3차
...,...,...,...,...,...,...,...
17527,서울특별시,양천구,신월동,NaN,1147010300,A10024872,목동 센트럴 아이파크위브 아파트
17585,서울특별시,양천구,신정동,NaN,1147010100,A10023886,호반써밋목동아파트
18119,서울특별시,양천구,목동,NaN,1147010200,A10024355,목동세종그랑시아 아파트
18616,서울특별시,양천구,신정동,NaN,1147010100,A10024398,래미안 목동아델리체


In [63]:
YC_APT_CODE = list(YC_GU["kaptCode"])

In [64]:
len(YC_APT_CODE)

131

In [65]:
url = "http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo"

In [67]:
dict_apt_list = []

for idx, apt_code in enumerate(YC_APT_CODE):
    params = {
        "serviceKey": getRTMSDataSvcAptTradeDev_de,
        "kaptCode": apt_code
    }

    response = requests.get(url, params=params)
    try:
        dict_content = xmltodict.parse(response.content)["response"]["body"]["item"]
        dict_apt_list.append(dict_content)
    except Exception as e:
        print(e)
        print(idx, apt_code)
    # df = pd.DataFrame(dict_content)

    # df.to_csv(f"data/서울특별시_양천구_아파트매매/서울특별시_양천구_아파트매매_{month}.csv",
    #           encoding="utf8",
    #           index=False
    # )

In [69]:
df = pd.DataFrame(dict_apt_list)
df.to_csv("data/서울특별시_양천구_공동주택기본정보.csv",
          encoding="utf8",
          index=False
)